In [1]:
# !git clone https://github.com/mnielsen/neural-networks-and-deep-learning.git ./data

Cloning into './data'...
remote: Enumerating objects: 1163, done.
remote: Total 1163 (delta 0), reused 0 (delta 0), pack-reused 1163
Receiving objects: 100% (1163/1163), 20.42 MiB | 455.00 KiB/s, done.
Resolving deltas: 100% (577/577), done.


In [33]:
import mnist_loader
import numpy as np

In [34]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

FileNotFoundError: [Errno 2] No such file or directory: '../data/mnist.pkl.gz'

In [29]:
def sigmoid(z):
    """ Computes the sigmoid, if param is vector or np array, then automatically vectorizes it"""
    return 1.0 / (1.0 + np.exp(-z))
def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [30]:
class Network:
    """ This class Represents the whole Neural Netwrk"""
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.rand(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feed_forward(self, a):
        """ Return the output of the network. shape of a is (n, 1) ndarray and 
        not (n,) vector."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """ Trains the NN using mini-batch stochastic gradient descent. The training data
        is a list of tuples "(x, y)" representing the training input with desired outputs."""
        
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f"Epoch {j}: {self.evaluate(test_data)} / {n_test}")
            else:
                print("Epoch {j} complete")
                
    def update_mini_batch(self, mini_batch, eta):
        """Updates the network's weights and biases by applying gradient descent using 
        backpropagation to a sinlge mini batch."""
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y) # Backprop is a fast way to compute the gradient
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_b = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        self.weights = [w - (eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        """Return a tuple (nabla_b, nabla_w) representing the gradient for the cost function C_x.
        nabla_b and nabla_w are layer by layer lists of numpy arrays"""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # store all the activations layer by layer
        zs = [] # to store the z-values
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Here -1 represents the last value
        
        for l in xrange(2, self.num_layers):
            z = zs[-1]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-1] = delta
            nabla_w[-1] = np.dot(delta, activations[-l-1].transpose())
            
        return (nabla_b, nabla_w)
    
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives for the output activations"""
        return (output_activations - y)
            

In [5]:
n = Network([2,3,1])